### Домашнее задание по булеву поиску

Задание выполнил Корнеев Алексей.

In [1]:
from collections import defaultdict
import gzip

Посмотрим на содержание файлов.

In [2]:
%%time
for i in range(8):
    # filename = 'C:/Users/apkor/Desktop/Учеба/4 семестр/Инфопоиск/Boolean_search/lenta_ru_dump' + \
    filename = 'dumps' + \
        '/dump_' + str(i + 1) + '.gz'
    data = []
    with gzip.open(filename) as file:
        data.extend(file.read().decode('utf-8', errors = 'ignore').lower().split())
    print(data[:100])

['z\x10\x00\x00', '*http://lenta.ru/news/2009/06/17/procedure/\x1a', '07:26,', '17', 'июня', '2009', 'генпрокуратура', 'подготовила', 'документы', 'для', 'экстрадиции', 'чичваркина', 'генпрокуратура', 'рф', 'начала', 'процедуру', 'экстрадиции', 'из', 'великобритании', 'бывшего', 'совладельца', 'компании', '"евросеть"', 'евгения', 'чичваркина,', 'пишет', 'в', 'среду', 'газета', '"', 'коммерсант', '".', 'по', 'данным', 'издания,', 'недавно', 'российское', 'ведомство', 'посетили', 'представители', 'правоохранительных', 'органов', 'великобритании,', 'с', 'которыми', 'был', 'согласован', 'перечень', 'необходимых', 'документов.', 'бумаги', 'уже', 'переведены', 'на', 'английский', 'язык', 'и', 'вскоре', 'будут', 'направлены', 'в', 'лондон.', 'чичваркин,', 'который', 'обвиняется', 'в', 'причастности', 'к', 'похищению', 'человека', 'и', 'вымогательству,', 'был', 'заочно', 'арестован', 'басманным', 'судом', 'в', 'январе', '2009', 'года,', 'напоминает', 'издание.', 'вместе', 'ним', 'по', 'делу', 

### Предобработка данных

Токенизируем текст и удалим лишние символы, сохраним список url и множество всех слов.

In [3]:
def data_preprocessing(path):
    """
    Предобработка сырых данных.
    """
    data = []
    num_of_files = 8
    for i in range(num_of_files):
        filename = path + '/dump_' + str(i + 1) + '.gz'
        with gzip.open(filename) as file:
            data.extend(file.read().decode('utf-8', errors = 'ignore').lower().split())
    
    words = set(data) # список всех слов в корпусе текстов
    urls = dict() # список всех сайтов
    buf = []
    
    def __parse_url(url):
        """
        Удаление лишних символов.
        """
        clear_url = ''
        ord_point = 46 # = ord('.')
        for c in url:
            clear_url += c
            if len(clear_url) == 4 and clear_url != 'http':
                clear_url = clear_url[1:]
            if ord(clear_url[-1]) < ord_point and 'http' in clear_url: # url кончился, появился лишний символ
                # break
                return clear_url[:-1]
        return clear_url[:-1]
   

    for word in data:
        if 'http://lenta' in word: # ссылки на lenta.ru могут находиться на самих страницах lenta.ru
            if len(buf) > 2:
                urls[__parse_url(buf[0])] = buf[1:]
                words.remove(buf[0])
            # buf.clear()
            buf = []
        buf.append(word)
    
    return words, urls    

In [4]:
%%time
# words, urls = data_preprocessing('C:/Users/apkor/Desktop/Учеба/4 семестр/Инфопоиск/Boolean_search/dumps')
words, urls = data_preprocessing('dumps')

CPU times: user 2.62 s, sys: 92.4 ms, total: 2.71 s
Wall time: 2.7 s


In [5]:
%%time
word_to_id = { word : id_ for (id_, word) in enumerate(words) }    # словарь { слово : id слова}
id_to_url = { id_ : url for (id_, url) in enumerate(urls.keys()) } # словарь { id документа : url сайта }
url_to_id = { url : id_ for (id_, url) in enumerate(urls.keys()) } # словарь { url сайта : id документа }

CPU times: user 135 ms, sys: 3.94 ms, total: 139 ms
Wall time: 138 ms


In [6]:
WORD_TO_ID_LEN = len(word_to_id)
len(word_to_id), len(id_to_url), len(url_to_id)

(284157, 9876, 9876)

### Сжатие (Simple 9)

In [7]:
class Simple9:
    def __init__(self):
        self.shifts = [1, 2, 3, 4, 5, 7, 9, 14, 28]
        self.masks = [i * (1 << 28) for i in range(1, 10)]
        self.encode_type = \
            [ [ self.shifts[-i - 1], 2**self.shifts[i] - 1, self.masks[-i - 1], self.shifts[i] ] \
            for i in range(len(self.masks)) ]
        self.decode_type = \
            { self.masks[-i - 1] : [ self.shifts[-i - 1], 2**self.shifts[i] - 1, self.shifts[i] ] \
            for i in range(len(self.masks)) }
        
    def simple9_encode(self, content):
        """
        Кодирование списка элементов content.
        """
        offset = 0
        res = []
        while offset < len(content):
            for encode_type in self.encode_type:
                (n, upper, code, single_shift) = encode_type
                if offset + n <= len(content) and max(content[offset:offset + n]) <= upper:
                    cur_val = content[offset]
                    for i in range(1, n):
                        cur_val |= (content[offset + i] << (i * single_shift))
                    offset += n
                    res.append(code | cur_val)
                    break
        return res
    
    def simple9_decode(self, content):
        """
        Декодирование списка элементов content.
        """
        res = []
        for item in content:
            (n, upper, single_shift) = self.decode_type[0xf0000000 & item]
            data = 0x0fffffff & item
            
            for i in range(n):
                res.append(upper & data)
                data >>= single_shift
        
        return res

In [8]:
%%time

reversed_index = defaultdict(list)
for id_, (url, words_list) in enumerate(urls.items()):
    for word in words_list:
        reversed_index[word_to_id[word]].append(url_to_id[url])

CPU times: user 2.78 s, sys: 32 ms, total: 2.82 s
Wall time: 2.82 s


In [9]:
%%time
from copy import deepcopy # для передачи списков в ф-ции

data = []
for key, value in reversed_index.items():
    list_copy = deepcopy(value)
    for i in range(len(list_copy) - 1, 0, -1):
        list_copy[i] -= list_copy[i - 1]
    tmp_list = [key, len(list_copy)] + list_copy # [ id, n, 1st item, 2nd item, ..., nth item ]
    data.extend(tmp_list) # id, n, 1st item, 2nd item, ..., nth item, id, n, 1st item, 2nd item, ..., nth item, ...

CPU times: user 3.25 s, sys: 27.8 ms, total: 3.28 s
Wall time: 3.28 s


In [10]:
%%time

simple9_encoder_decoder = Simple9()
data_compressed = simple9_encoder_decoder.simple9_encode(data)

CPU times: user 8.77 s, sys: 68.4 ms, total: 8.84 s
Wall time: 8.87 s


На этом этапе происходит процесс передачи данных.

In [11]:
%%time

data_decompressed = simple9_encoder_decoder.simple9_decode(data_compressed)

CPU times: user 1.39 s, sys: 56 ms, total: 1.45 s
Wall time: 1.48 s


In [12]:
assert(len(data) == len(data_decompressed))
for i in range(len(data)):
    assert(data[i] == data_decompressed[i])

Теперь проверим, что восстановленный по сжатым данным обратный индекс идентичен исходному.

In [13]:
%%time

reversed_index_decoded = defaultdict(list)
flag = 0 # тип декодируемого элемента:
         # 0 - id слова
         # 1 - n
         # 2 - список id документов
for item in data_decompressed:
    if flag == 0:
        key = item
        flag += 1
    elif flag == 1:
        n = item
        cur_list = []
        flag += 1
    else:
        n -= 1
        if len(cur_list) > 0:
            cur_list.append(item + cur_list[-1])
        else:
            cur_list.append(item)
        
        if n == 0:
            reversed_index_decoded[key] = deepcopy(cur_list)
            flag = 0


CPU times: user 4.97 s, sys: 56 ms, total: 5.03 s
Wall time: 5.03 s


In [14]:
assert(len(reversed_index) == len(reversed_index_decoded))
for i in range(len(reversed_index)):    
    assert(reversed_index[i] == reversed_index_decoded[i])

Алгоритм сжатия работает корректно.

In [15]:
del reversed_index_decoded # удаляем ненужную копию обратного индекса

### Булев поиск

На последнем этапе необходимо выполнить несколько задач:
1. Реализовать потоковую обработку дерева запросов.
2. Перевести запрос в обратную польскую нотацию.

In [16]:
nodes_idx = set() # множество индексов вершин дерева поиска

In [17]:
from random import randint

def search(query, id_to_url):
    """
    Поиск документов, удовлетворяющих булевому запросу.
    """
    global nodes_idx
    search_tree = dict()
    
    def __streaming_tree_processing(search_tree, root, reversed_index, word_to_id):
        """
        Потоковая обработка дерева.
        """
        
        def __binary_search(list_, key, left = 0, right = -100): # можно еще подумать над выбором константы,
                                                                 # 100 кажется оптимальным значением
            """
            Бинарный поиск.
            """
            if right == -100:
                right = len(list_) - 1
            if left == len(list_):
                return left
            if left > right:
                return left
            mid = ((left + right) // 2)
            if list_[mid] < key:
                return __binary_search(list_, key, mid + 1, right)
            elif list_[mid] > key:
                return __binary_search(list_, key, left, mid - 1)
            return mid

        idx_res = -1
        res = []
        
        def __run_search(ptr, idx_res):
            """
            Рекурсивный обход дерева.
            """
            node = search_tree[ptr]
            if node[0] == 'binary':
                if node[1] == '&':
                    return max(
                        __run_search(node[2], idx_res),
                        __run_search(node[3], idx_res)
                    )
                if node[1] == '|':
                    return min(
                        __run_search(node[2], idx_res),
                        __run_search(node[3], idx_res)
                    )
            
            elif node[0] == 'unary':
                if node[1] == '!':
                    tmp = __run_search(node[2], idx_res)
                    return (tmp + 1) if tmp == idx_res else idx_res
            
            elif node[0] == 'operand':
                docs = reversed_index[node[1]]
                node[2] = __binary_search(docs, idx_res, left = node[2])
                if node[2] == len(docs):
                    return float('inf')
                elif docs[node[2]] >= idx_res:
                    return docs[node[2]]
            
        while idx_res < max_idx_res:
            query_out = __run_search(root, idx_res)
            if idx_res == query_out:
                res.append(idx_res)
                idx_res += 1
            else:
                if idx_res < query_out:
                    idx_res = query_out
        return res
    
    def __create_node(stack, item, item_type):
        new_idx = __get_unique_idx()
        if item_type == 'binary':
            node = { new_idx : [ item_type, item, stack[-2], stack[-1] ] }
            del stack[-2:]
        elif item_type == 'unary':
            node = { new_idx : [ item_type, item, stack[-1] ] }
            stack.pop()
        elif item_type == 'operand':
            node = { new_idx : [ item_type, item, 0 ] }
            
        stack.append(new_idx)
        search_tree.update(node)
        
        return stack
    
    
    def __transform_query(query):
        """
        Преобразование запроса в список строк трех типов: \
        скобки, операнды и операции над множествами (унарные и бинарные).
        """
        res = []
        token = ""
        
        for c in query.lower():
            if c.isalpha(): # часть операнда
                token += c
            else:
                if token.strip(): # операнд считан целиком, лишние символы удалены
                    res.append(token)
                token = ""
                if c.strip(): # операция над множествами
                    res.append(c)
        if token.strip(): # последний/-ая операнд / операция над множеством
            res.append(token)
        return res
    
    def __reverse_polish_notation(query_list):
        """
        Перевод запроса в обратную польскую нотацию.
        """
        rpn = []
        stack = []
        priorities = {
            '(' : 1,
            '|' : 2,
            '&' : 3,
            '!' : 4
        } # приоритеты операций над множествами
        
        for item in query_list:
            if item == '(':
                stack.append('(')
            elif item in priorities.keys():
                while len(stack) > 0 and priorities[stack[-1]] >= priorities[item]:
                    rpn.append(stack[-1])
                    stack.pop()
                stack.append(item)
            elif item == ')':
                while stack[-1] != '(':
                    rpn.append(stack[-1])
                    stack.pop()
                stack.pop()
            else:
                rpn.append(item)
        
        rpn.extend(reversed(stack))
        
        return rpn
    
    def __get_unique_idx():
        # global nodes_idx
        
        new_idx = randint(0, WORD_TO_ID_LEN)
        while new_idx in nodes_idx:
            new_idx = randint(0, WORD_TO_ID_LEN)
        nodes_idx.add(new_idx)
        return new_idx
    
    # получаем query_list
    query_list = __transform_query(query)
    
    rpn = __reverse_polish_notation(query_list)
    
    stack = []
    
    for item in rpn:
        if item == '!':
            stack = __create_node(stack, item, 'unary')
        elif item == '&' or item == '|':
            stack = __create_node(stack, item, 'binary')
        else:
            stack = __create_node(stack, word_to_id[item], 'operand')
    
    root = stack[0]
    
    max_idx_res = max(url_to_id.values())
    doc_idx = __streaming_tree_processing(search_tree, root, reversed_index, word_to_id)
    output = []
    for id_ in doc_idx:
        output.append(id_to_url[id_])
    
    nodes_idx = set()
    return output

In [18]:
query = 'Россия & США & Китай & !(Индия | Япония)'

In [19]:
%%time

result = search(query, id_to_url)

print('Found', len(result), 'result(s).')
print(*result, sep = '\n')

Found 13 result(s).
http://lenta.ru/articles/2015/02/14/liberman/
http://lenta.ru/articles/2015/12/16/rulers/
http://lenta.ru/news/2005/07/15/china1/
http://lenta.ru/articles/2015/08/20/ustrussia/
http://lenta.ru/articles/2015/01/27/solar/
http://lenta.ru/articles/2015/04/03/viet/
http://lenta.ru/news/2008/02/12/lavrov/
http://lenta.ru/articles/2010/03/30/review/
http://lenta.ru/columns/2015/03/09/strategy/
http://lenta.ru/sport/2003/07/18/soccer/
http://lenta.ru/articles/2006/05/04/resolution/
http://lenta.ru/articles/2002/08/08/weapon/
http://lenta.ru/articles/2013/03/20/sipri/
CPU times: user 2.78 ms, sys: 3.98 ms, total: 6.76 ms
Wall time: 6.02 ms


Сайты из выдачи поискового алгоритма проверил вручную, все они подходят под булев запрос.